In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Get the same results each time
np.random.seed(0)

# Load the training data
data = pd.read_csv("data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

**Задание 1: Разделение данных на train и test**

In [3]:
X_train, X_test, y_train, y_test = train_test_split(comments, target, test_size=0.3, random_state=42)

**Задание 2: Преобразование текста в числовой формат**

In [4]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

**Задание 3: Обучение модели логистической регрессии**

In [5]:
lr = LogisticRegression(max_iter=2000)
lr.fit(X_train_vec, y_train)
print(f'Accuracy: {round(accuracy_score(y_test, lr.predict(X_test_vec)), 2)}')

Accuracy: 0.93


**Задание 4: Функция для предсказания токсичности**

In [14]:
def func_predict(comment):
    return round(lr.predict_proba(vectorizer.transform([comment]))[0][1], 2)

**Задание 5: Тестирование модели на примерах**

In [15]:
print("'Apples are stupid':", func_predict("Apples are stupid"))
print("'I love apples':", func_predict("I love apples"))

'Apples are stupid': 1.0
'I love apples': 0.06


**Задание 6: Вывод самых токсичных слов**

In [16]:
feature_names = vectorizer.get_feature_names_out()
coef = lr.coef_[0]
top_toxic_word = coef.argsort()[-10:][::-1]

print('10 самых токсичных слов:')
for i in top_toxic_word:
    print(f'{feature_names[i]}: {round(coef[i], 3)}')

10 самых токсичных слов:
stupid: 9.225
idiot: 8.719
idiots: 8.445
stupidity: 7.533
idiotic: 6.814
crap: 6.569
dumb: 6.441
pathetic: 6.412
hypocrite: 6.374
moron: 6.351


**Задание 7: Анализ токсичных слов**

Вызывают ли у вас удивление какие-нибудь из них? - Нет, типичные английские ругательства, а какие не напрямую ругательные, то так или иначе с негативным подтекстом слова

**Задание 8: Тестирование на предвзятость**

In [18]:
list_comments = [
    'I have a christian friend',
    'I have a muslim friend',
    'I have a white friend',
    'I have a black friend'
]

for comment in list_comments:
    print(f"'{comment}': {round(func_predict(comment), 2)}")

'I have a christian friend': 0.19
'I have a muslim friend': 0.51
'I have a white friend': 0.4
'I have a black friend': 0.59


***Результаты демонстрируют явную предвзятость модели, причем в нескольких аспектах:***
  
1. Религиозный:
Христианство: _0.19_ (низкий риск токсичности)  

Ислам: _0.51_ (высокий риск, почти в 3 раза выше)  

Это отражает исламофобию, присутствующую в исходных данных  

2. Расовый:

"White friend": _0.40_  

"Black friend": _0.59_ (на 48% выше)

Модель ассоциирует упоминание чернокожих с токсичностью сильнее  

3. Абсурдность абсолютных значений:

Нейтральные фразы формата "I have a X friend" вообще не должны классифицироваться как токсичные

**Задание 9: Тип предвзятости**

Это пример исторической предвзятости (historical bias), где существующие в обществе стереотипы и предубеждения отражаются в данных. Модель учится на этих данных и воспроизводит те же предубеждения

**Задание 10: Идеи для улучшения**

1. Сразу напрашивается балансировка данных, чтобы обеспечить равное представление разных групп в обучающих данных

2. Удаление токсичных примеров с расистскими/религиозными стереотипами, чтобы не тянуть их в модель

3. Оценивать не отдельные слова, а общий смысл комментария